In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 통계 분석
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, chi2_contingency, f_oneway
from scipy.stats import mannwhitneyu, fisher_exact, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import ast
# import pingouin as pg
# import scikit_posthocs as sp

import platform

In [3]:
def display_set(col = None ,row = None, col_wid = None):
# 데이터 프레임 열,행, 행 내 글자수 생략 기준 설정
    warnings.filterwarnings('ignore')
    pd.set_option('display.max_columns', col)
    pd.set_option('display.max_rows', row)
    pd.set_option('display.max_colwidth', col_wid)

def kor_font_set(os):
    # os별 한글 폰트 설정 함수
 
    if os == 'Windows':
        plt.rcParams['font.family'] = 'Malgun Gothic'
    elif os == 'Darwin':  # macOS
        plt.rcParams['font.family'] = 'AppleGothic'
    else:  # Linux
        plt.rcParams['font.family'] = 'NanumGothic'
print("한글 폰트 설정 완료!")
    

def plt_figrue_regular(width,height):
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['figure.figsize'] = (width, height)

한글 폰트 설정 완료!


In [4]:
# 열,행, 행내 글자수 생략 함수
display_set(100,100,200)

# OS별 한글폰트 자동 세팅
kor_font_set(platform.system())

# plt 사이즈 세팅 (가로,세로)
plt_figrue_regular(12,8)

In [ ]:
def read_csv(filename):
    # 아이템,챔프 정보 데이터셋 호출
    data = pd.read_csv(filename,index_col=0)
    return data

filename = 'TFT_Match_info.csv'

df = read_csv(filename)

,gameId,gameDuration,level,lastRound,Ranked,ingameDuration,entry,combination,main_comb1,main_comb2,sub_comb,tier,Champion1,star1,items1,Champion2,star2,items2,Champion3,star3,items3,Champion4,star4,items4
0,KR_4247538593,2142.470703,8,35,1,2134.272217,"['XinZhao', 'Rakan', 'JarvanIV', 'Sona', 'Lulu', 'Neeko', 'Karma', 'Soraka']","['Protector', 'Set3_Mystic', 'Set3_Celestial', 'DarkStar', 'StarGuardian', 'Rebel', 'TemplateTrait']",Protector 4,Set3_Mystic 4,Set3_Celestial 3,Challenger,XinZhao,3,"[69, 25, 25]",Rakan,3.0,"[37, 69]",JarvanIV,3.0,[27],Sona,3.0,[46]
1,KR_4247538593,2142.470703,9,35,2,2134.272217,"['Yasuo', 'Kayle', 'MissFortune', 'AurelionSol', 'Malphite', 'Sona', 'Jinx', 'Soraka', 'MasterYi']","['Rebel', 'Set3_Blademaster', 'Blaster', 'Set3_Mystic', 'Valkyrie', 'Mercenary', 'Set3_Brawler', 'StarGuardian', 'Starship']",Rebel 6,Set3_Blademaster 3,Blaster 2,Challenger,Yasuo,2,"[99, 22, 23]",Kayle,2.0,"[26, 49, 49]",MissFortune,1.0,"[99, 26, 12]",AurelionSol,2.0,"[37, 66]"
2,KR_4247538593,2142.470703,8,34,3,2073.459229,"['KaiSa', 'Shaco', 'Rumble', 'Annie', 'Ekko', 'Karma', 'Lux', 'Fizz']","['Infiltrator', 'DarkStar', 'MechPilot', 'Set3_Sorcerer', 'Cybernetic', 'Demolitionist', 'Set3_Mystic', 'Valkyrie']",Infiltrator 4,DarkStar 3,MechPilot 3,Challenger,KaiSa,2,"[99, 2, 23]",Shaco,2.0,"[16, 19, 55]",Rumble,3.0,"[36, 45]",Annie,2.0,[37]
3,KR_4247538593,2142.470703,8,33,4,1998.146729,"['Shaco', 'Fizz', 'KaiSa', 'Gangplank', 'Annie', 'Rumble', 'MissFortune', 'Ekko']","['Infiltrator', 'MechPilot', 'Demolitionist', 'Mercenary', 'Valkyrie', 'Blaster', 'Cybernetic', 'DarkStar', 'Set3_Sorcerer', 'SpacePirate', 'TemplateTrait']",Infiltrator 4,MechPilot 3,Demolitionist 2,Challenger,Shaco,2,"[15, 16, 14]",Fizz,2.0,"[99, 46, 55]",KaiSa,2.0,"[44, 37]",Gangplank,2.0,"[37, 36]"
4,KR_4247538593,2142.470703,9,33,5,1986.443237,"['AurelionSol', 'Gangplank', 'MissFortune', 'MasterYi', 'Lulu', 'Ziggs', 'Yasuo', 'Sona', 'Jinx']","['Rebel', 'Blaster', 'Demolitionist', 'Mercenary', 'Set3_Blademaster', 'Set3_Mystic', 'Set3_Celestial', 'SpacePirate', 'Starship', 'Valkyrie']",Rebel 6,Blaster 2,Demolitionist 2,Challenger,AurelionSol,2,"[46, 36, 44]",Gangplank,1.0,"[49, 33, 15]",MissFortune,2.0,"[12, 79]",MasterYi,2.0,[5]


In [16]:
def Convert_case(df,collist):
    
    # 전적 데이터 셋 챔피언 컬럼별 소문자로 변환 후
    df[collist] = (
        df[collist]
        .apply(lambda x: x.str.lower())
    )

    # 타이틀로 재변환
    df[collist] = (
        df[collist]
        .apply(lambda x: x.str.title())
    )
    return df


# 변환 할 데이터 프레임
case_df = df
# 변환 할 컬럼명
case_col = ['entry','Champion1', 'Champion2', 'Champion3', 'Champion4']

df = Convert_case(case_df,case_col)